In [3]:
import pandas as pd
import os
from swarm import Swarm, Agent
from duckduckgo_search import DDGS
from datetime import datetime
from dotenv import load_dotenv
load_dotenv()


import logging
logging.basicConfig(level=logging.INFO)


In [ ]:
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("OPENAI_MODEL_NAME")
base_url = os.getenv("OPENAI_BASE_URL")


In [5]:
current_date = datetime.now().strftime("%Y-%m")

# Initialize Swarm client 
client = Swarm()

# Create Internet Search Tool
def get_news_articles(topic):
    print(f"Running DuckDuckGo search for {topic}")

    # Initialize DuckDuckGo search client
    ddgs = DDGS()

    # Search for news articles
    search_results = ddgs.text(f'{topic} {current_date}', max_results=5)

    if search_results:
        news_results = "\n\n".join([f"Title: {result['title']}\nURL: {result['href']}\nDescription: {result['body']}" for result in search_results])
        return news_results
    else:
        return f"Not able to find any news articles for {topic} on {current_date}"



In [71]:
# Create AI Agents

# News agent that will fetch news
news_agent = Agent(
    name="News Agent",
    instructions="You provide the latest news articles for the given topic using DuckDuckGo search",
    functions=[get_news_articles],
    model='llama3.2:latest'
)

# Editor agent that will edit & summarize the news
editor_agent = Agent(
    name="Editor Agent",
    instructions="You provide the edited  version of the news articles so that they can be published. Each story in separate sections.",
    model='llama3.2:latest'
)


In [72]:
# Create the swarm workflow

def run_news_workflow(topic):
    print(f"Running news workflow for {topic}")

    news_resposne = client.run(
        agent=news_agent,
        messages=[{"role": "user", "content": f"Fetch top news articles about '{topic}' as of {current_date}. Provide titles, URLs, and short summaries."}]
    )

    if not news_resposne.messages:
        raise Exception("Failed to fetch news. Please try again.")


    news = news_resposne.messages[-1]["content"]
    print(f"News fetched for {topic}:\n{news}")

    editor_response = client.run(
        agent=editor_agent,
        messages=[{"role": "user", "content": news}]
    )

    edited_news = editor_response.messages[-1]["content"]

    return edited_news

In [ ]:
print('\n\n Publish ready version \n\n',run_news_workflow("AI"))